In [2]:
!pip install openai python-dotenv python-docx

  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached python_docx-1.1.2-py3-none-any.whl (244 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 23.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 20.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from docx import Document

In [4]:
class ReqDoc:
    def __init__(self, file_path):
        self.file_path = file_path

    def extract(self):
        """
        Reads the content of a .docx file and returns the paragraphs as a list of strings.
        """
        try:
            # Check if the file exists
            if not os.path.exists(self.file_path):
                raise FileNotFoundError(f"The file {self.file_path} was not found.")

            # Attempt to open and read the document
            doc = Document(self.file_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
            return text

        except FileNotFoundError as fnf_error:
            print(fnf_error)
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None


In [6]:
# Initialize and constants
#If Running in Google Colab, set up env variable as per the instructions in README.md file.

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [7]:
#Set up system prompt for extracting just the requirements from the document

req_doc_system_prompt = "You are provided with a complete requirements specifications document. \
You are able to decide which content from that document are related to actual requirements, identify each requirement as \
functional or non-functional and list them all.\n"
req_doc_system_prompt += "You should respond in JSON as in this example:"
req_doc_system_prompt += """
{
    "requirements": [
        {"RequirementNo": "FR-01", "Requirement Description": "description of this functional requirement goes here"},
        {"RequirementNo": "FR-02": "Requirement Description": "description of this functional requirement goes here"},
        {"RequirementNo": "NFR-01": "Requirement Description": "description of this non-functional requirement goes here"},
        {"RequirementNo": "NFR-02": "Requirement Description": "description of this non-functional requirement goes here"}
    ]
}
"""

In [8]:
#Set up user prompt, sending in the requirements doc as input and calling the ReqDoc.extract function. Key to note here is the explicit instructions to
#respond in JSON format.

def req_doc_user_prompt(doc):
    user_prompt = "Here is the contents from a requirement document.\n"
    user_prompt += f"{doc.extract()} \n"
    user_prompt += "Please scan through the document and extract only the  actual requirements. For example, ignore sections or \
paragraphs such as Approvers, table of contents and similar sections which are not really requirements.\
You must respond in a JSON format"
    user_prompt = user_prompt[:25_000] # Truncate if more than 25,000 characters
    return user_prompt
    
    

In [9]:
#Function to call chatgpt-4o-mini model with the user and system prompts set above and returning the json formatted result obtained from chatgpt

def get_requirements(doc):
    reqdoc = ReqDoc(doc)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": req_doc_system_prompt},
            {"role": "user", "content": req_doc_user_prompt(reqdoc)}
        ],
      response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_requirements("requirements.docx")

The file requirements.docx was not found.


{'requirements': []}

### Next, we will make another call to gpt-4o-mini

In [12]:
#Set up system prompt to ask for test cases in table format

system_prompt = "You are an assitant that receives a list of functional and non functional requirements in JSON format. You are the expert in generating unit test cases for each requirement. \
You will create as many different test cases as needed for each requirement and produce a result in a table. Order the table by requirement No. Provide clear details on test case pass criteria. \
The table will contain the following columns. \
1.S No\
2.Requirement No\
3.Requirement Description\
4.Test Case ID\
5.Test case summary\
6.Test case description\
7.Success criteria \n"

In [13]:
# Set up user prompt passing in the req doc file. This in turn will call the get_requirements function, which will make a call to chatgpt.

def get_testcase_user_prompt(reqdoc):
    user_prompt = "You are looking at the following list of requirements. \n"
    user_prompt += f"{get_requirements(reqdoc)}\n"
    user_prompt += "Prepare unit test cases for each of these requirements in a table and send that table as response. \n"
    user_prompt += user_prompt[:25000]
    return user_prompt

In [14]:
#This is the 2nd call to chatgpt to get test cases. display(Markdown) will take care of producing a neatly formatted table output.
def create_testcase_doc(reqdoc):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_testcase_user_prompt(reqdoc)}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
#The final piece of code. Ensure the requirements doc file is in the same path where the code is running and provide that filename instead.
create_testcase_doc("reqdoc.docx")

Here is a table with unit test cases for each of the provided requirements:

| S No | Requirement No | Requirement Description                                              | Test Case ID | Test Case Summary                           | Test Case Description                                                                                | Success Criteria                           |
|------|----------------|---------------------------------------------------------------------|---------------|---------------------------------------------|-----------------------------------------------------------------------------------------------------|-------------------------------------------|
| 1    | FR-01          | Users can submit feedback via an online form.                      | TC_FR01_001   | Valid feedback submission                   | Verify that a user can submit valid feedback through the online form.                              | Feedback is successfully recorded in the system. |
| 2    | FR-01          | Users can submit feedback via an online form.                      | TC_FR01_002   | Invalid feedback submission                 | Verify that a user receives an error message on submitting invalid feedback.                        | Error message is displayed as expected.                 |
| 3    | FR-01          | Users can submit feedback via an online form.                      | TC_FR01_003   | Empty feedback submission                   | Verify that a user cannot submit the feedback form if no content is provided.                       | Error message specific to empty form is shown.       |
| 4    | FR-02          | Feedback can be categorized by type (e.g., product, service).      | TC_FR02_001   | Validate feedback categorization           | Verify that feedback submitted can be categorized correctly into predefined types.                  | Feedback is categorized correctly.           |
| 5    | FR-02          | Feedback can be categorized by type (e.g., product, service).      | TC_FR02_002   | Invalid category selection                  | Verify that an error is thrown when an invalid category is selected during feedback submission.     | Error message for invalid category is shown.       |
| 6    | FR-03          | Administrators can generate detailed reports.                      | TC_FR03_001   | Generate feedback report                    | Verify that an administrator can generate a feedback report successfully.                          | Report is generated and contains correct data.     |
| 7    | FR-03          | Administrators can generate detailed reports.                      | TC_FR03_002   | Empty report request                        | Verify the behavior when an administrator requests a report without any filters or parameters.     | Appropriate message regarding empty filters.     |
| 8    | NFR-01         | The system must support at least 10,000 concurrent users.           | TC_NFR01_001  | Test concurrent user load                   | Simulate a load of 10,000 users accessing the system concurrently.                                 | System remains responsive and functional.       |
| 9    | NFR-01         | The system must support at least 10,000 concurrent users.           | TC_NFR01_002  | Test with 10,001 users                      | Simulate a load of 10,001 users accessing the system concurrently.                                 | System indicates it cannot support beyond 10,000.  |
| 10   | NFR-02         | The feedback portal must load within 3 seconds for all pages.      | TC_NFR02_001  | Validate page load time                     | Measure the load time for multiple pages in the feedback portal.                                   | All pages load in less than 3 seconds.    |
| 11   | NFR-02         | The feedback portal must load within 3 seconds for all pages.      | TC_NFR02_002  | Stress test Pages load time                 | Measure page load time when simulated with high concurrency (e.g., 100 users).                     | All pages load in less than 3 seconds.    |
| 12   | NFR-03         | Data must be encrypted during transmission and at rest.             | TC_NFR03_001  | Validate data encryption in transmission    | Verify that data is encrypted while being sent from the user device to the server.                  | Data captured in transit is encrypted.       |
| 13   | NFR-03         | Data must be encrypted during transmission and at rest.             | TC_NFR03_002  | Validate data encryption at rest            | Verify that stored data in the database is encrypted.                                             | Data in the database appears as encrypted.       |

This table summarizes the unit test cases devised for each requirement while ensuring clarity on the test case descriptions and success criteria.